In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from urllib.request import urlretrieve
import re

# 뉴스 스크래핑

In [ ]:
posts = []
news_category = {
    100: '정치',
    101: '경제',
    102: '사회',
    103: '생활/문화',
    104: '세계',
    105: 'IT/과학',
}
for i in range(100, 106):
    category = news_category.get(i)
    url =f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={i}#&date=%2000:00:00&page=1'
    driver = webdriver.Chrome()
    driver.get(url)
    post_list = driver.find_elements(By.XPATH, '//*[@id="section_body"]/ul/li')
    for k in post_list:
        post = k.find_element(By.XPATH, 'dl/dt[2]/a')
        news_title = post.text
        link = post.get_attribute('href')
        try:
            news_image = k.find_element(By.XPATH, 'dl/dt[1]/a/img')
            news_image_url = news_image.get_attribute('src')
            match = re.search(r'(\d+).jpg', news_image_url)
            if match:
                news_image_name = match.group(1) + '.jpg'
            else:
                news_image_name = ''  # 추출 실패 시 빈 문자열로 설정
            print(news_image_name)
            news_image = news_image_url.replace('https', 'http')
            news_image_path = f'/static/images/news_image/{news_image_name}'
            urlretrieve(news_image, f'C:/Users/user/PycharmProjects/final_project_web/static/images/news_image/{news_image_name}')
        except:
            news_image_path = ''
        driver.execute_script(f"window.open('{link}', '_blank');")
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(1)
        create_dt=driver.find_element(By.XPATH, '//*[@id="ct"]/div[1]/div[3]/div[1]/div/span').get_attribute('data-date-time')
        news_name=driver.find_element(By.XPATH,'//*[@id="ct"]/div[1]/div[1]/a/img[1]').get_attribute('title')
        post_origin = driver.find_element(By.XPATH, '//*[@id="dic_area"]')
        post_origin_text = post_origin.text
        posts.append([create_dt, news_name, category, news_image_path, news_title, post_origin_text, link ])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    driver.quit()

# 네이버 뉴스 데이터 DB 저장

In [ ]:
import psycopg2
from datetime import datetime
# 여러분의 데이터베이스 연결 정보
db_connection = psycopg2.connect(
    host="localhost",
    dbname='final_project',
    user='postgres',
    password="sesac0916",
    port=5432
)
# 데이터베이스 커서 생성
cursor = db_connection.cursor()
for news_item in posts:
    try:
        cursor.execute("""
            INSERT INTO final_project.news_information
            (create_dt, media_company, news_genre, news_image_path, news_title, news_origin, news_url, news_summary)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (news_url) DO NOTHING;
        """, (news_item[0], news_item[1], news_item[2], news_item[3], news_item[4], news_item[5], news_item[6], ''))
    except Exception as e:
        print(f"Error inserting news_item: {news_item}")
        print(f"Error message: {e}")
    time.sleep(0.3)
# 변경사항을 저장
db_connection.commit()
# 연결 종료
cursor.close()
db_connection.close()

# 파인튜닝한 뉴스 요약 모델에 DB에 저장된 데이터 불러와 재저장

In [ ]:
import psycopg2
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
# 파인 튜닝한 모델 가져오기
saved_model_path = r"C:\Users\user\Downloads\bart_summarization_model_v2-20231222T004142Z-001\bart_summarization_model_v2"
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained(saved_model_path)
# 요약 함수 설정
def summarize(text):
    inputs = tokenizer.encode_plus(text, return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], length_penalty=1.0, num_beams=4, max_length=250, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
# 나의 postgresql 연결하기
db_connection = psycopg2.connect(
    host="localhost",
    dbname='final_project',
    user='postgres',
    password="sesac0916",
    port=5432
)
cursor = db_connection.cursor()
# 뉴스 요약문이 빈 행의 id와 본문 만 가져오기
cursor.execute("SELECT id, news_origin FROM final_project.news_information WHERE news_summary = ''")
rows = cursor.fetchall()
for row in rows:
    news_id, news_text = row
    summary = summarize(news_text)  # 뉴스 요약 함수로 요약 실행
    # 요약된 뉴스를 id가 같은 행에 news_summary 업데이트 하기
    cursor.execute(
        "UPDATE final_project.news_information SET news_summary = %s WHERE id = %s",
        (summary, news_id)
    )
# 변경 사항 커밋 및 연결 종료
    db_connection.commit()
cursor.close()